In [1]:
with open("C:/Users/GenITeam/Downloads/rumi_poetry11.txt", "r", encoding="utf-8") as f:

    text = f.read()

print("Number of characters:", len(text))

Number of characters: 40469


In [2]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"',-.0123456789:;?ABCDEFGHIJKLMNOPRSTUVWYabcdefghijklmnopqrstuvwxyz~–‘’“”…
77


In [3]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode(text[:7]))
print(decode(encode(text[:7])))

[49, 64, 57, 54, 57, 58, 66]
funknow


In [ ]:
import torch 
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([40469]) torch.int64
tensor([49, 64, 57, 54, 57, 58, 66, 57,  1, 48, 67, 52, 62, 63, 48, 57, 46, 48,
         0, 64, 57, 47, 52, 62, 46, 58, 65, 48, 61, 48, 47,  1, 45, 48, 44, 64,
        63, 68,  0, 63, 51, 44, 63,  4, 62,  1, 51, 58, 66,  1, 68, 58, 64,  1,
        44, 61, 48,  0, 62, 58,  1, 49, 44, 61,  0, 58, 57, 48,  1, 47, 44, 66,
        57,  0, 53, 64, 62, 63,  1, 55, 52, 54, 48,  1, 44,  1, 62, 64, 57,  0,
        61, 52, 50, 51, 63,  1, 49, 61, 58, 56,  1, 66, 52, 63, 51, 52, 57,  0,
        68, 58, 64,  1, 66, 52, 55, 55,  1, 44, 61, 52, 62, 48,  0,  0,  0, 29,
        34,  1, 39, 28, 29, 38,  1, 25, 21, 37, 39, 28,  0,  0, 52, 57,  1, 63,
        51, 52, 62,  1, 48, 44, 61, 63, 51,  0, 52, 57,  1, 63, 51, 52, 62,  1,
        48, 44, 61, 63, 51,  0, 52, 57,  1, 63, 51, 52, 62,  1, 52, 56, 56, 44,
        46, 64, 55, 44, 63, 48,  1, 49, 52, 48, 55, 47,  0, 66, 48,  1, 62, 51,
        44, 55, 55,  1, 57, 58, 63,  1, 59, 55, 44, 57, 63,  1, 44, 57, 68,  1,
        

In [ ]:
batch_size = 16 
block_size = 32 
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0


torch.manual_seed(1337)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]


def get_batch(split):
   
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)

print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    
    xb, yb = get_batch('train')

    
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.211277 M parameters
step 0: train loss 4.5221, val loss 4.5157
step 100: train loss 2.6904, val loss 2.7120
step 200: train loss 2.5028, val loss 2.5623
step 300: train loss 2.4036, val loss 2.5170
step 400: train loss 2.3106, val loss 2.4296
step 500: train loss 2.2104, val loss 2.3451
step 600: train loss 2.1343, val loss 2.2677
step 700: train loss 2.0846, val loss 2.2278
step 800: train loss 2.0346, val loss 2.1784
step 900: train loss 1.9741, val loss 2.1571
step 1000: train loss 1.9372, val loss 2.1013
step 1100: train loss 1.8847, val loss 2.1124
step 1200: train loss 1.8573, val loss 2.0902
step 1300: train loss 1.8349, val loss 2.0794
step 1400: train loss 1.8003, val loss 2.0545
step 1500: train loss 1.7778, val loss 2.0457
step 1600: train loss 1.7518, val loss 2.0544
step 1700: train loss 1.7112, val loss 2.0357
step 1800: train loss 1.6997, val loss 2.0241
step 1900: train loss 1.6706, val loss 2.0195
step 2000: train loss 1.6459, val loss 2.0112
step 2100: train loss 1.

In [ ]:
while True:
    prompt = input("Enter a starting line (or 'exit' to stop): ")
    if prompt.lower() == 'exit':
        break
    try:
        context = torch.tensor([encode(prompt)], dtype=torch.long, device=device)
        generated = m.generate(context, max_new_tokens=200)
        print("\n📝 Generated Text:\n")
        print(decode(generated[0].tolist()))
        print("\n" + "-"*50 + "\n")
    except KeyError as e:
        print(f"❌ Character '{e.args[0]}' not in vocabulary. Try a simpler prompt.")



📝 Generated Text:

THE IS SUN IS Rulon Put youre, who not longing souls
a new beloved and not for herst,
Det with mout around
drunk for to


I AS YOWNE HEART

come no be wind on this
cometentann
grone and der
bushis dole
how could i 

--------------------------------------------------


📝 Generated Text:

THE EARTH IS READY

lomove mind and depart accord,
Droped cond, and the soul?

There of my heart, I dead I, without your door the been the dancing
but ving to of a trave
full changed over moral me
to be all
the do

--------------------------------------------------

❌ Character 'X' not in vocabulary. Try a simpler prompt.
❌ Character 'X' not in vocabulary. Try a simpler prompt.
